In [1]:
import os
import polars as pl
import altair as alt

import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [2]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~pl.col('komora').is_in(["se","psse"])
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).collect()

In [3]:
df.select(pl.col("komora_komplet")).unique()

komora_komplet
str
"""NS RČS 1954-1960"""
"""Federální shromáždění ČSFR, 1992 – 1992 Sněmovna lidu"""
"""Federální shromáždění ČSSR, 1986 – 1990 Sněmovna lidu"""
"""Federální shromáždění ČSFR, 1990 – 1992 Sněmovna lidu"""
"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998"""
"""Federální shromáždění ČSFR, 1992 – 1992 Společné schůze SL a SN"""
"""Federální shromáždění ČSSR, 1981 – 1986 Sněmovna lidu"""
"""Federální shromáždění ČSSR, 1981 – 1986 Sněmovna národů"""
"""Federální shromáždění ČSFR, 1990 – 1992 Sněmovna národů"""


## Rychlé sčoty

In [5]:
df.group_by_dynamic(index_column="datum",every="1w").agg(pl.col("pocet_slov").sum()).sort(by="pocet_slov",descending=True).head(10)

datum,pocet_slov
date,u32
2011-10-31,507439
2023-02-27,478349
1930-11-24,388685
1926-11-22,371876
1927-11-28,341179
1928-10-22,330268
1921-11-21,311333
2023-07-10,310767
1923-11-26,308507


In [6]:
df.group_by_dynamic(index_column="datum",every="1m").agg(pl.col("pocet_slov").sum()).sort(by="pocet_slov",descending=True).head(10)

datum,pocet_slov
date,u32
2023-03-03,131371
2025-04-16,128315
2023-10-18,126715
1927-05-06,121229
2011-11-02,121073
1991-12-05,120345
2022-02-02,120047
1926-06-17,119990
2024-01-24,117239


In [7]:
alt.Chart(
    df.group_by_dynamic(index_column="datum",every="3m").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1300
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [8]:
graf_obdobi_93 = alt.Chart(
    df.group_by("obdobi").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("obdobi:N"),
    alt.Y("pocet_slov:Q")
)

graf_obdobi_93

alt.Chart(...)

In [9]:
import os
try:
    os.makedirs("grafy")
except:
    pass

In [10]:
graf_obdobi_93.save(os.path.join("grafy","graf_obdobi_93.png"))

In [11]:
slov_na_den = df.group_by("obdobi").agg(pl.col('datum').max()).rename({'datum':'do'}).join(
    df.group_by("obdobi").agg(pl.col('datum').min()).rename({'datum':'od'}),
    how='left',on='obdobi'
).join(
    df.group_by("obdobi").agg(pl.col("pocet_slov").sum()),
    how='left',on='obdobi'
).with_columns(
    (pl.col('do') - pl.col('od')).alias('trvani').dt.total_days()
).with_columns(
    (pl.col('pocet_slov') / pl.col('trvani')).alias('slov_na_den')
).with_columns(
    pl.lit(0).alias("nula")
).with_columns(
    pl.when(
        pl.col("obdobi") == 2021
    ).then(
        pl.lit("soucasne")
    ).otherwise(
        pl.lit("davne")
    ).alias("barvicky")
)

In [12]:
slov_na_den

obdobi,do,od,pocet_slov,trvani,slov_na_den,nula,barvicky
i32,date,date,u32,i64,f64,i32,str
1968,1968-12-21,1968-07-10,141769,164,864.445122,0,"""davne"""
1971,1976-09-21,1971-12-07,1778253,1750,1016.144571,0,"""davne"""
1986,1990-05-18,1986-06-12,2335930,1436,1626.692201,0,"""davne"""
1998,2002-06-13,1998-07-15,8963383,1429,6272.486354,0,"""davne"""
2013,2017-10-16,2013-11-25,11227634,1421,7901.220267,0,"""davne"""
2010,2013-08-20,2010-06-22,6563568,1155,5682.742857,0,"""davne"""
1992,1992-12-30,1992-06-25,1235285,188,6570.664894,0,"""davne"""
1918,1920-04-15,1918-11-14,2490805,518,4808.503861,0,"""davne"""
1948,1954-10-27,1948-06-10,1862714,2330,799.448069,0,"""davne"""


In [64]:
slov_po_snemovnach = alt.Chart(
    slov_na_den.to_pandas(),
    width=330,
    height=120,
    title = alt.Title(
        "Průměrný počet slov pronesených za den ve Sněmovně",
        subtitle=[
            'Počítají se všechny dny od prvního do posledního zasedání, nejenom',
            'dny jednací. Šířka sloupců odpovídá reálné délce funkčního období.',
            'V letech 1945-1992 počítáme dohromady všechny zákonodárné sbory',
            's výjimkou Slovenské národní rady.'
        ]
    )
).mark_bar(
).encode(
    alt.X("od:T", axis=alt.Axis(title=None)),
    alt.X2("do:T"),
    alt.Y("nula:Q", axis=alt.Axis(title=None, orient="right",format="2r")),
    alt.Y2("slov_na_den:Q"),
    alt.Color(
        "barvicky:N",
        scale=alt.Scale(range=['#6B8EAA','#D55F50']),
        legend=None
    )
).configure_axisX(
    grid=False, domain=False
).configure_axisY(
    grid=True, domain=False
).configure_view(
    stroke='transparent'
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="shared"
)

slov_po_snemovnach

alt.Chart(...)

In [66]:
me_to_neurazi(
    graf=slov_po_snemovnach, 
    soubor='slov_po_snemovnach', 
    kredity='vizualizace: iROZHLAS.cz | zdroj projevů: psp.cz', 
    slozka_na_serveru='stenoprotokoly'
)

<figure>
    <a href="https://data.irozhlas.cz/stenoprotokoly/slov_po_snemovnach.svg" target="_blank">
    <img src="https://data.irozhlas.cz/stenoprotokoly/slov_po_snemovnach.svg" width="100%" alt="Graf s titulkem „Průměrný počet slov pronesených za den ve Sněmovně“. Další texty by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [15]:
help(me_to_neurazi)

Help on function me_to_neurazi in module src.me_to_neurazi:

me_to_neurazi(graf: altair.vegalite.v5.api.LayerChart, kredity: str, soubor: str, slozka='grafy', zvetseni=1.5, slozka_na_serveru='grafy')



In [16]:
df.filter(pl.col('obdobi') == 1996).filter(pl.col('datum') > pl.date(2004, 1, 1))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32


## Pro srovnání: Babička

In [18]:
import requests
from bs4 import BeautifulSoup

In [19]:
babicka = requests.get("https://web2.mlp.cz/koweb/00/03/34/76/44/babicka.html").text
babicka = BeautifulSoup(babicka).text.split("Dávno, dávno již tomu")[1].split("Vydala Městská knihovna v Praze")[0]

In [20]:
babicka.count(" ")

67169

In [21]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov").sum()).item() / babicka.count(" ")

233.15544373148327

## Dlouhé x krátké projevy

Cílem studie bylo najít krátký a jednoduchý test k měření psychomotorického tempa, který by se dal použít v klinické praxi. Metodika: Bylo vyšetřeno 63 zdravých dobrovolníků průměrného stáří 28,6 ± 12,5 let (rozsah 15–71 let). Byl měřen počet slov za jednu minutu spontánního monologu (měřítko psychomotorického tempa), čas nutný k vyjmenování 12 zvířat (zkrácená verze sémantické fluence), 12 slov začínajících na písmeno K (zkrácená verze fonemické fluence), čas potřebný k vyjmenování měsíců v roce, k hlasitému počítání od 101 do 110 a jiné krátké testy. Výsledky: Během spontánního monologu bylo zaznamenáno 105,7 ± 38,7 slov za minutu. Byla nalezena negativní korelace mezi spontánní řečí a rychlostí počítání (r = -0,69; p < 0,001). Vyjmenování 12 měsíců bylo více než 3× rychlejší než vyjmenování 12 zvířat a 7× rychlejší než vyjmenování 12 slov na K.

Zdroj: [PSYCHOMOTORICKÉ TEMPO, RYCHLOST ŘEČI A MYŠLENÍ RYCHLOST ŘEČI A MYŠLENÍ](https://www.psychiatriepropraxi.cz/pdfs/psy/2007/05/06.pdf)

Bacha: i dlouhý projev může být přerušený krátkou vsuvkou.

In [25]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov").sum()).item() 

15660818

In [26]:
df.filter(pl.col("obdobi") == 2021).select(pl.col("pocet_slov").sum()).item() / (100 * 60 * 40)

65.25340833333334

In [27]:
alt.Chart(
    df.filter(pl.col("pocet_slov") <= 100).group_by_dynamic(index_column="datum",every="3m").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [28]:
alt.Chart(
    df.filter(pl.col("pocet_slov") >= 1000).group_by_dynamic(index_column="datum",every="3m").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

## Kolik toho není přepsáno?

In [30]:
neprepsane = pl.read_parquet("data/neprepsane_schuze.parquet").filter(pl.col('komora') == 'ps')

In [31]:
neprepsane

komora,obdobi,schuze,cast,soubor,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,i32,i32,i32,str,date,str,str,bool,bool
"""ps""",2013,9,9,"""2013ps_stenprot_009schuz_s009009.htm""",null,null,null,false,null
"""ps""",2013,10,306,"""2013ps_stenprot_010schuz_s010306.htm""",null,null,null,false,null
"""ps""",2013,23,944,"""2013ps_stenprot_023schuz_s0231944.htm""",null,null,null,false,null
"""ps""",2013,23,945,"""2013ps_stenprot_023schuz_s0231945.htm""",null,null,null,false,null
"""ps""",2013,23,946,"""2013ps_stenprot_023schuz_s0231946.htm""",null,null,null,false,null
"""ps""",2013,23,947,"""2013ps_stenprot_023schuz_s0231947.htm""",null,null,null,false,null
"""ps""",2013,23,948,"""2013ps_stenprot_023schuz_s0231948.htm""",null,null,null,false,null
"""ps""",2013,23,949,"""2013ps_stenprot_023schuz_s0231949.htm""",null,null,null,false,null
"""ps""",2013,23,950,"""2013ps_stenprot_023schuz_s0231950.htm""",null,null,null,false,null


In [32]:
df.group_by("obdobi").agg(pl.col("soubor").unique().len()).sort(by="soubor").rename({"soubor":"prepsanych"}).join(
    neprepsane.group_by("obdobi").agg(pl.col("soubor").unique().len()).sort(by="obdobi").rename({'soubor':"neprepsanych"}),
    how='left', on='obdobi'
).with_columns(
    (pl.col('prepsanych') + pl.col('neprepsanych')).alias('celkem')
).with_columns(
    (pl.col('neprepsanych') / pl.col('celkem')).alias('podil_neprepsanych')
)

obdobi,prepsanych,neprepsanych,celkem,podil_neprepsanych
i32,u32,u32,u32,f64
1968,39,null,null,null
1945,408,null,null,null
1960,409,null,null,null
1992,458,null,null,null
1954,497,null,null,null
1969,539,null,null,null
1964,678,null,null,null
1976,685,null,null,null
1981,722,null,null,null


## Přibudou ještě?

In [34]:
df.filter(pl.col("obdobi") == 2017).filter(pl.col("datum") > pl.date(2021,9,1)).select(pl.col("pocet_slov")).sum()

pocet_slov
u32
214637


In [35]:
df.filter(pl.col("obdobi") == 2017).select(pl.col("datum").max())

datum
date
2021-10-05


In [68]:
df.select(pl.col("pocet_slov").sum())

pocet_slov
u32
134061250


In [74]:
df.select(pl.col("pocet_slov").sum()).item() / (105 * 60 * 8 * 365.25)

7.2825337070716945